# Batch Training 

- epoch = 1 forward and backward pass of ALL training samples
- batch_size = number of training samples in one forward & backward pass
- number of iterations = number of passes, each pass using [batch_size] number of samples
- eg. 100 samples, batch_size=20 --> 100/20 = 5 iterations in 1 epoch

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

In [4]:
import pandas as pd
wine = pd.read_csv('./wine.data',  header=None)
wine.to_csv('wine.csv', index=None)

In [5]:
wine.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


![image-3](image-3.png)


In [6]:
# implementing custom dataset
class Wine(Dataset):
    
    def __init__(self):
        # data loading
        xy = np.loadtxt('wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.x = torch.from_numpy(xy[:,1:])
        self.y = torch.from_numpy(xy[:,[0]]) #n_samples, 1
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        # dataset[0]
        return self.x[index], self.y[index]
        
    def __len__(self):
        # len(dataset)
        return self.n_samples

In [7]:
# initialize Wine class object
dataset = Wine()
first_data = dataset[0]
first_data

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]),
 tensor([1.]))

In [10]:
# DataLoader object creation
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True, num_workers=2) 

dataloader

In [13]:
# iterator
dataiter = iter(dataloader)
data = next(dataiter)
data

[tensor([[1.1820e+01, 1.7200e+00, 1.8800e+00, 1.9500e+01, 8.6000e+01, 2.5000e+00,
          1.6400e+00, 3.7000e-01, 1.4200e+00, 2.0600e+00, 9.4000e-01, 2.4400e+00,
          4.1500e+02],
         [1.2770e+01, 3.4300e+00, 1.9800e+00, 1.6000e+01, 8.0000e+01, 1.6300e+00,
          1.2500e+00, 4.3000e-01, 8.3000e-01, 3.4000e+00, 7.0000e-01, 2.1200e+00,
          3.7200e+02],
         [1.4210e+01, 4.0400e+00, 2.4400e+00, 1.8900e+01, 1.1100e+02, 2.8500e+00,
          2.6500e+00, 3.0000e-01, 1.2500e+00, 5.2400e+00, 8.7000e-01, 3.3300e+00,
          1.0800e+03],
         [1.3030e+01, 9.0000e-01, 1.7100e+00, 1.6000e+01, 8.6000e+01, 1.9500e+00,
          2.0300e+00, 2.4000e-01, 1.4600e+00, 4.6000e+00, 1.1900e+00, 2.4800e+00,
          3.9200e+02]]),
 tensor([[2.],
         [2.],
         [1.],
         [2.]])]

since batch_size=4, the DataLoader loads 4 features and 4 labels at each iteration. 

In [15]:
# training loop
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)
print(total_samples, n_iterations)

178 45


So, 45 iterations in each epoch

# Data Transforms

In [23]:
# implementing custom dataset
class Wine(Dataset):
    
    def __init__(self, transform=None):
        # data loading
        xy = np.loadtxt('wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        
        # note that we don't convert to tensor here
        self.x = xy[:,1:]
        self.y = xy[:,[0]] #n_samples, 1
        self.n_samples = xy.shape[0]
        
        self.transform = transform
        
    def __getitem__(self, index):
        # dataset[0]
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
        
    def __len__(self):
        # len(dataset)
        return self.n_samples

In [24]:
# create transform class
# ToTensor class which converts to tensor

class ToTensor:
    def __call__(self,sample):
        inputs, labels = sample
        return torch.from_numpy(inputs), torch.from_numpy(labels)

In [25]:
# create Wine class object
dataset = Wine(transform=ToTensor()) #<-- transform variable has ToTensor class object

In [26]:
first_data = dataset[0]
first_data

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]),
 tensor([1.]))

The data is transformed into Tensor.